импорты

In [1]:
import os
import numpy as np
from natsort import natsorted
import matplotlib
import matplotlib.pyplot as plt
import re
import pandas as pd
from scipy import signal

константы

In [2]:
# 153.8336486816406250 884.2712384333286764    измеряемый диапазон. 0-2136 диапазон данных
start_nm = 400  # нм
end_nm = 700  # нм

x = pd.read_excel("x points nm.xlsx")
x = np.array(x["nm"].values[1 : len(x) - 1])


def nm_to_x(nm):
    return xint.index(nm) + 1


def x_to_nm(index):
    return x[index]


# номера точек в массиве по координате х
for i in range(len(x)):
    if x[i] > start_nm:
        start_x = i
        break
for i in range(len(x) - 1, 0, -1):
    if x[i] < end_nm:
        end_x = i
        break
x = x[start_x:end_x]
xint = list(np.array(x, dtype=int))
tail_x = nm_to_x(650)
head_x = nm_to_x(475)

функции получения данных

In [3]:
def get_data_rmr(path):
    """получение всей папки в виде списка с массивами np"""
    _file_list = np.array(natsorted(os.listdir(path)))
    _data = []
    for file in range(len(_file_list)):
        with open(path + _file_list[file], "r", encoding="utf8") as spec:
            spec = re.split(",", spec.read())
            _data.append(np.array(spec[start_x:end_x], dtype=float))
    #    _data.append(get_rmr(spec))
    return _data

получение данных из rmr. папка с папками

In [4]:
# получение всех спектров в виде списка в котором
# каждая строка - папка, а каждый элемент - массив np с спектром

data = []  # сглаженные графики
smooth = True
main_folder = r"C:\Users\Nik\Desktop\t"
main_folder = main_folder.replace(chr(92), "/")
folders_list = list(natsorted(os.listdir(main_folder)))
print(main_folder)
print(folders_list)

import concurrent.futures

data_row = [0] * len(folders_list)  # изанчальные графики


def func(fl):
    _ind = folders_list.index(fl)
    data_row[_ind] = get_data_rmr(main_folder + "/" + fl + "/")
    print(fl, " ", _ind + 1, " recived ", len(data_row[_ind]))


with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(func, folders_list)


if smooth == True:
    for i in range(len(data_row)):
        d = []
        for j in range(len(data_row[i])):
            d.append(signal.savgol_filter(data_row[i][j], 60, 3))
        data.append(d)
else:
    data = data_row.copy()
print("получено " + str(len(data_row)) + " папок")

C:/Users/Nik/Desktop/t
['bad', 'good']
bad   1  recived  4000
good   2  recived  4300
получено 2 папок


добавить 1 папку

In [4]:
data = []
data_row = []
folders_list = []

In [6]:
# получение всех спектров в виде списка в котором
# каждая строка - папка, а каждый элемент - массив np с спектром
smooth = True
main_folder = r"D:\Data\скр2\points\p1\1_2"
main_folder = main_folder.replace(chr(92), "/")
folders_list = np.append(folders_list, "bad")
print(main_folder)
print(folders_list)


data_row.append(get_data_rmr(main_folder + "/"))
# print("data from " + str(folders_list[i]) + " recived")
print("получено " + str(len(data_row)) + " папок")

if smooth == True:
    for i in range(len(data_row)):
        d = []
        for j in range(len(data_row[i])):
            d.append(signal.savgol_filter(data_row[i][j], 60, 3))
        data.append(d)
else:
    data = data_row.copy()

D:/Data/скр2/points/p1/1_2
['good' 'bad']
получено 2 папок


функции выборки максимума и разницы

In [ ]:
# функции для получения данных
def dmax(mas: np.array):
    return np.max(mas)


def dmax_650(mas: np.array):
    return np.max(mas) - mas[tail_x]


def dmax_list(mas: list):
    _max = []
    for a in mas:
        _max.append(a[nm_to_x(550)])
    return _max


def dmax_650_list(mas: list):
    _max = []
    for i in range(len(mas)):
        _max.append(mas[i][nm_to_x(550)] - mas[i][tail_x])
    return _max


def zero_index(mas):
    _zero = []
    for i in range(len(mas)):
        if np.max(mas[i]) < 0.1:
            _zero.append(i)
    return _zero

таймлайн

In [ ]:
_folder = 0
_a = 400
print_grafs = True
k = len(data[_folder])

_num = 0
for _b in range(0, 13000, _a):

    if len(data[_folder]) < _b + _a:
        _a = len(data[_folder]) - _b
    _mas = data[_folder][_b : _b + _a]  # 100 cпектров из папки
    # for i in range(len(_mas)):
    #    _mas[i] = signal.savgol_filter(_mas[i], 60, 3)

    # plt.subplots(figsize=[10, 10]);plt.clf()
    fig, axs = plt.subplots(3, 1, figsize=(9, 5), dpi=150)
    # fig.tit title("синтез " + str(folders_list[_folder]))
    fig.suptitle(
        "Измерение " + str(folders_list[_folder]) + " " + str(_b) + "-" + str(_b + _a)
    )

    _sp = 0
    for i in range(len(_mas)):
        axs[0].plot(np.arange(_sp, _sp + len(_mas[i])), _mas[i], lw=1)
        _sp += len(_mas[i])
    # axs[0].set_ylim(-0.05, 0.5)
    axs[0].set_ylabel("Интенсивность, отн.ед.")
    # axs[0].set_xticklabels(np.arange(0,_sp),np.arange(0,30))
    # axs[0].axes.tick_params(axis='x',which='minor',direction='out',bottom=True,length=80)

    axs[1].set_ylabel("I(max), отн.ед.")
    minus = dmax_list(_mas)
    for i in range(len(_mas)):
        axs[1].scatter(i, minus[i], s=5)
    axs[1].plot(np.arange(len(minus)), minus, lw=0.2, color="black")
    # axs[1].set_ylim(-0.05, 0.5)

    axs[2].set_ylabel("I(max)-I(650), отн.ед.")
    minus = dmax_650_list(_mas)
    for i in range(len(_mas)):
        axs[2].scatter(i, minus[i], s=5)
    axs[2].plot(np.arange(len(minus)), minus, lw=0.2, color="black")

    if print_grafs == True:
        plt.savefig("t/" + str(_num) + ".png")
    _num += 1

# fig.clf();fig.clear()

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

выборка лучших спектров

In [8]:
def data_for_hist(mas, ind):
    ar = np.zeros(len(mas))
    if ind == "max":
        for i in range(len(mas)):
            ar[i] = np.max(mas[i][nm_to_x(520) : nm_to_x(600)])
    else:
        for i in range(len(mas)):
            ar[i] = mas[i][ind]
    return ar


print_graf = True
fol = "aa"
data_new = []
gap = 0.02
for n in range(len(folders_list)):
    masiv = data[n]
    massiv_row = data_row[n]
    _d = []
    ar_max = data_for_hist(masiv, "max")
    ar_mean = data_for_hist(masiv, nm_to_x(650))
    ar_max_mean = ar_max - ar_mean

    maximum = np.max(ar_max_mean)
    ar = []  # индексы спектров с наибольшей разницей между мах и 650

    num_bins = 20  # количество интервалов
    range_values = (maximum / 2) / num_bins
    # границы интервалов
    bin_edges = np.arange(maximum / 2, maximum, range_values)
    # середины интервалов
    bins = bin_edges[:-1] + range_values / 2
    hist, _ = np.histogram(ar_max_mean, bins=bin_edges)
    zero = bin_edges[np.argmax(hist) + 1]
    if print_graf == True:
        fig, ax = plt.subplots(figsize=(5, 5), dpi=300)
        plt.ioff()
        ax.hist(
            ar_max_mean, bin_edges, alpha=0.6, color="blue", edgecolor="k", linewidth=1
        )

        ax.scatter(
            bin_edges[np.argmax(hist[5:20])] + 5.5 * range_values,
            np.max(hist[5:20]),
            color="red",
        )

        ax.plot(bins, hist, color="red", lw=2)
        plt.title('I(max)-I(650) histogram of'+str( folders_list[n]))
        plt.ylabel("Count, a.u.")
        plt.xlabel("Intensity, a.u.")
        plt.xticks(rotation=90)
        ax.set_xticks(bin_edges)

        plt.tight_layout()

        plt.savefig(fol + "/" + folders_list[n] + ".png")
    maximum = bin_edges[np.argmax(hist[5:20])] + 5.5 * range_values
    for i in range(len(ar_max_mean)):
        if maximum - range_values < ar_max_mean[i] < maximum + range_values:
            ar.append(i)
    # if len(ar) == 0:
    # continue
    ar2 = []  # значения интенсивнотей в точке max
    for i in range(len(ar)):
        ar2.append(np.max(masiv[ar[i]]))
    minimum = np.min(ar2)

    ar3 = []  # индексы более подходящих спектров
    for i in range(len(ar)):
        if np.max(masiv[ar[i]]) < minimum + gap:
            ar3.append(ar[i])
    for i in range(len(ar3)):
        _d.append(massiv_row[ar3[i]])
    data_new.append(_d)

In [ ]:
fig = plt.figure(figsize=(8, 8), dpi=900)
# projection='3d'


from sklearn.preprocessing import normalize

# normalize matrix by rows


ax = plt.axes()
mas = data[0][0 : len(data[0])]
for i in range(len(mas)):
    if np.max(mas[i]) > 0.175:
        mas[i] = np.zeros(len(mas[i]))
for i in ar3:
    mas[i][0:50] = np.zeros(50)
plt.rcParams["figure.autolayout"] = False
z = mas
z = np.array(z)
z = z.T

X = np.arange(len(mas))
Y = np.arange(len(mas[0]))

# Задаем сетку точек (x, y)
X, Y = np.meshgrid(X, Y)

# Отображаем поверхность
gr = ax.pcolor(Y, X, z, cmap="turbo", alpha=1)
ax.set_xticks(np.arange(0, 875, int(875 / 6)))
ax.set_xticklabels(np.arange(400, 701, 50))
# axs[0].axes.tick_params(axis='x',which='minor',direction='out',bottom=True,length=80)
ax.set_xlabel("Wawelenght, nm")

# Можно также установить заголовки по осям Y и/или показать график
ax.set_ylabel("Spectra count")


axins = inset_axes(
    ax,
    width="4%",
    height="100%",
    loc="lower left",
    bbox_to_anchor=(1.05, 0.0, 1, 1),
    bbox_transform=ax.transAxes,
    borderpad=0,
)
plt.colorbar(gr, cax=axins)

plt.tight_layout()
plt.savefig("map4.png")
plt.show()

In [6]:
spec = []  # сглаженный
spec_row = []  # не сглаженный


for i in range(len(data_new)):
    _a = len(data_new[i])

    _b = np.sum(data_new[i], axis=0)

    _b /= _a

    spec.append(signal.savgol_filter(_b, 100, 3))
    spec_row.append(_b)

построение лучших спектров

In [ ]:
import matplotlib as mpl

In [ ]:
# построение красивых итоговых графиков
cmap = matplotlib.colormaps.get("Set1")(np.linspace(0, 1, len(spec)))
plt.figure(figsize=(12, 6), dpi=400)
k = 0
for n in range(len(spec)):
    plt.plot(
        x,
        spec[n],
        lw=0.8,
        color=cmap[n],
        label=str(folders_list[n])
        + " "
        + str(round(len(data_new[n]) * 100 / len(data[n]), 2))
        + "%",
    )
plt.xlabel("Длинна волны,нм")
plt.ylabel("Интенсивность, отн.ед.")
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
plt.savefig("3(2).png")

In [12]:
for n in range(len(spec)):
    plt.figure(figsize=(5, 5), dpi=300)

    plt.ioff()
    for i in range(len(data[n])):
        if x_to_nm(np.argmax(data[n][i])) > 520:
            plt.plot(
                x,
                data[n][i],
                lw=0.1,
                alpha=0.8,
                color="black",
            )
        else:
            plt.plot(
                x,
                data[n][i],
                lw=0.1,
                alpha=0.8,
                color="tab:Green",
            )
    for i in range(len(data_new[n])):
        plt.plot(
            x,
            signal.savgol_filter(data_new[n][i], 60, 3),
            lw=0.03,
            alpha=0.5,
            color="blue",
        )
    plt.plot(x, spec[n], color="red")
    plt.title(
        folders_list[n]
        + ", "
        + str(round(len(data_new[n]) * 100 / len(data[n]), 2))
        + "%"
    )
    plt.scatter(1, 1, color="tab:Green", label="Oil spectra")
    plt.scatter(1, 1, color="black", label="Reagent spectra")
    plt.scatter(1, 1, color="tab:blue", label="Selected spectra")
    plt.scatter(1, 1, color="tab:red", label="Average spectrum")

    plt.ylim(-0.01, 1.3 * np.max(spec[n]))
    plt.xlim(400, 700)
    plt.xlabel("Wavelenght, nm")
    plt.ylabel("Absorbance, a.u.")
    plt.title(str(len(data[n])) +' spectra of '+str(folders_list[n])+' droplet series')
    plt.legend(loc='lower left')
    plt.tight_layout()
    plt.savefig("aa\\s" + str(folders_list[n]) + ".png")
    # matplotlib.pyplot.close(figure)

получение смапа

In [ ]:
cmap = matplotlib.colormaps.get("Set1")(np.linspace(0, 1, len(spec)))

дескрипторы

In [ ]:
def coef_parabola(sp):
    """коэффициенты параболы"""
    _m = np.max(sp)
    _gap = 0.005
    _i = np.argmax(sp)
    if _i == 0 or _i + 300 > len(sp):
        return [0]
    _p, _q = 0, 0
    for i in range(_i, _i + 300):
        if sp[i] < _m - _gap:
            _q = i
            break
    for i in range(_i, _i - 300, -1):
        if sp[i] < _m - _gap:
            _p = i

            break
    try:
        _coefs = np.polyfit(
            x[_p:_q], sp[_p:_q], 2
        )  # Вторым аргументом передается степень полинома
    except:
        return [0]
    return _coefs


def w_h_75(sp):
    """ширина, высота"""
    _h = np.max(sp) - (np.max(sp) - sp[nm_to_x(450)]) * 0.25
    _start, _end = 0, 0
    _ind_max = np.argmax(sp)
    if _ind_max == 0 or _ind_max + 300 > len(sp):
        return [0, 0]
    for i in range(_ind_max, _ind_max + 300):
        if sp[i] <= _h:
            _end = i
            break
    for i in range(_ind_max, _ind_max - 300, -1):
        if sp[i] <= _h:
            _start = i
            break
    _w = x_to_nm(_end) - x_to_nm(_start)
    return (_w, _h, x_to_nm(_start), x_to_nm(_end))


def w_h_50(sp):
    """ширина, высота"""
    _h = np.max(sp) - (np.max(sp) - sp[nm_to_x(450)]) * 0.5
    _start, _end = 0, 0
    _ind_max = np.argmax(sp)
    if _ind_max == 0 or _ind_max + 300 > len(sp):
        return [0, 0]
    for i in range(_ind_max, _ind_max + 300):
        if sp[i] <= _h:
            _end = i
            break
    for i in range(_ind_max, _ind_max - 300, -1):
        if sp[i] <= _h:
            _start = i
            break
    _w = x_to_nm(_end) - x_to_nm(_start)
    return (_w, _h, x_to_nm(_start), x_to_nm(_end))


def angle_old(sp, point):
    _x1 = x_to_nm(np.argmax(sp))
    _y1 = np.max(sp)
    _x2 = point
    _y2 = sp[nm_to_x(point)]
    return ((_y2 - _y1) / (_x2 - _x1), _x1, _x2, _y1, _y2)


def angle(sp, point):
    _w = (650 - x_to_nm(np.argmax(sp))) / 4
    _x1 = x_to_nm(np.argmax(sp)) + _w
    _y1 = sp[nm_to_x(round(_x1))]
    _x2 = point
    _y2 = sp[nm_to_x(round(_x2))]
    return ((_y2 - _y1) / (_x2 - _x1), _x1, _x2, _y1, _y2)

In [ ]:
# выбор максимума
S_name = pd.Series(folders_list)
S_Imax = pd.Series(name="I(max)")
S_NMmax = pd.Series(name="I nm")
S_I650 = pd.Series(name="I(650)")
S_Imax_m_I650 = pd.Series(name="I(max) - I(650)")
S_Imax_d_I650 = pd.Series(name="I(max) d I(650)")
S_I400 = pd.Series(name="I(450)")
S_Imax_m_I400 = pd.Series(name="I(max) - I(450)")
S_Imax_d_I400 = pd.Series(name="I(max) d I(450)")
S_I400_d_I650 = pd.Series(name="I(400) d I(650)")
S_coef_a = pd.Series(name="coef A")
S_height75 = pd.Series(name="height75")
S_weight75 = pd.Series(name="weight75")
S_height50 = pd.Series(name="height50")
S_weight50 = pd.Series(name="weight50")
S_angle = pd.Series(name="ang")
_sp = nm_to_x(450)
for i in range(len(spec)):
    S_Imax.loc[i] = np.max(spec[i])
    S_NMmax.loc[i] = x[np.argmax(spec[i])]
    S_I650.loc[i] = spec[i][tail_x]
    S_Imax_m_I650.loc[i] = S_Imax.loc[i] - S_I650.loc[i]
    S_Imax_d_I650.loc[i] = S_Imax.loc[i] / S_I650.loc[i]
    S_I400.loc[i] = spec[i][_sp]
    S_Imax_m_I400.loc[i] = S_Imax.loc[i] - S_I400.loc[i]
    S_Imax_d_I400.loc[i] = S_Imax.loc[i] / S_I400.loc[i]
    S_I400_d_I650.loc[i] = S_I400.loc[i] / S_I650.loc[i]
    S_coef_a.loc[i] = coef_parabola(spec[i])[0]
    _w_h = w_h_75(spec[i])
    S_height75.loc[i] = _w_h[1]
    S_weight75.loc[i] = _w_h[0]
    _w_h = w_h_50(spec[i])
    S_height50.loc[i] = _w_h[1]
    S_weight50.loc[i] = _w_h[0]
    S_angle.loc[i] = angle(spec[i], 600)[0]

In [ ]:
for n in range(len(spec)):
    plt.figure(figsize=(10, 6))
    plt.ioff()
    plt.title(folders_list[n])
    # спектр
    plt.plot(x, spec[n], c="black")
    # максимум
    plt.scatter(S_NMmax[n], S_Imax[n], c="red", label="I(max) nm: " + str(S_NMmax[n]))
    plt.scatter(
        S_NMmax[n], S_Imax[n], c="red", label="I(max) : " + str(round(S_Imax[n], 5))
    )
    # 450
    plt.scatter(
        450,
        S_I400[n],
        c="blue",
        label="I(450) : "
        + str(round(S_I400[n], 5))
        + "\n"
        + "I(max) - I(450) : "
        + str(round(S_Imax_m_I400[n], 5))
        + "\n"
        + "I(max) d I(450) : "
        + str(round(S_Imax_d_I400[n], 5)),
    )
    # 650
    plt.scatter(
        650,
        S_I650[n],
        c="green",
        label="I(650) : "
        + str(round(S_I650[n], 5))
        + "\n"
        + "I(max) - I(650) : "
        + str(round(S_Imax_m_I650[n], 5))
        + "\n"
        + "I(max) d I(650) : "
        + str(round(S_Imax_d_I650[n], 5)),
    )
    # парабола
    coef = coef_parabola(spec[n])
    x1 = np.linspace(S_NMmax[n] - 15, S_NMmax[n] + 15, 100)
    plt.plot(
        x1,
        coef[0] * x1**2 + coef[1] * x1 + coef[2],
        alpha=0.9,
        c="crimson",
        label="coef a : " + str(round(coef[0], 8)),
    )
    # ширина на 75%высоты
    plt.plot(
        (w_h_75(spec[n])[2], w_h_75(spec[n])[3]),
        (S_height75[n], S_height75[n]),
        c="seagreen",
        label="weight75 : " + str(round(S_weight75[n], 3)),
    )
    # высота на 75%высоты
    plt.plot(
        (w_h_75(spec[n])[2], w_h_75(spec[n])[2]),
        (0, S_height75[n]),
        c="mediumblue",
        label="height75 : " + str(round(S_height75[n], 5)),
    )
    # ширина на 75%высоты
    plt.plot(
        (w_h_50(spec[n])[2], w_h_50(spec[n])[3]),
        (S_height50[n], S_height50[n]),
        c="springgreen",
        label="weight50 : " + str(round(S_weight50[n], 3)),
    )
    # высота на 75%высоты
    plt.plot(
        (w_h_50(spec[n])[2], w_h_50(spec[n])[2]),
        (0, S_height50[n]),
        c="purple",
        label="height50 : " + str(round(S_height50[n], 5)),
    )
    ang = angle(spec[n], 600)
    plt.plot(
        (ang[1], ang[2]),
        (ang[3], ang[4]),
        c="orange",
        label="angle : " + str(round(ang[0], 7)),
    )
    plt.legend(loc="upper right")
    plt.savefig("des/" + str(folders_list[n]) + ".png")

построение карт

In [ ]:
import sys

sys.path.append(r"C:\Users\Nik\Desktop\prog")
from pyfitit import *

In [ ]:
data_measure = pd.concat(
    [
        S_Imax,
        S_NMmax,
        S_I650,
        S_Imax_m_I650,
        S_Imax_d_I650,
        S_I400,
        S_Imax_m_I400,
        S_Imax_d_I400,
        S_coef_a,
        S_height75,
        S_weight75,
        S_height50,
        S_weight50,
        S_angle,
    ],
    axis=1,
)
data_measure.to_excel("output 4.2.xlsx", index=False)

In [ ]:
data_speed = pd.read_excel("s1.xlsx")
data_measure = pd.concat(
    [
        S_Imax,
        S_NMmax,
        S_I650,
        S_Imax_m_I650,
        S_Imax_d_I650,
        S_I400,
        S_Imax_m_I400,
        S_Imax_d_I400,
        S_coef_a,
        S_height75,
        S_weight75,
        S_height50,
        S_weight50,
        S_angle,
    ],
    axis=1,
)
data_table = pd.concat([data_speed, data_measure], axis=1)
print(data_table.columns[0:])

In [ ]:
data_table.to_excel("output 1.xlsx", index=False)

In [ ]:
data_table = data_table.drop(10)
print(data_table)

In [ ]:
df = pd.read_csv("params copy.csv", sep="\t")

In [ ]:
df = pd.read_excel("output test cut.xlsx")

In [ ]:
data_table = df
print(data_table)

In [ ]:
# 2D КАРТЫ
# import sklearn
data_table["имя"] = data_table["имя"].astype(str)
plotDescriptors2d(
    data_table,
    descriptorNames=["v АК", "v ЦН"],
    # labelNames=data_table.columns[5:],
    labelNames=[
        "I(max)",
        "I nm",
        "I(650)",
        "I(max)-I(650)",
        "I(400)",
        "I(max)-I(400)",
        "coef A",
        "height",
        "weight",
        "ang",
    ],
    textColumn=data_table.columns[0],
    cv_count=len(data_table),
    # unknown="unknown.params",
    folder_prefix="results/des",
    # model_regr=sklearn.linear_model.RidgeCV(),
)

In [ ]:
print(len(spec.pop(len(data_table))))

In [ ]:
sample = Sample(
    params=data_table,
    spectra=spec[0 : len(data_table)],
    energy=x,
    meta=dict(nameColumn="имя"),
)

sample.saveToFolder("sample")

In [ ]:
# data_table["имя"] = data_table["имя"].astype(str)
# sample = Sample(params=data_table, spectra=spec.pop(len(data_table)), energy=x, meta=dict(nameColumn="имя"))

# sample.saveToFolder("sample")
descriptor.plot_cv_result(
    sample,
    # textColumn=data_table.columns[0],
    features=["v АК", "v ХЗ", "v ЦН"],
    label_names=[
        "I(max)",
        "I nm",
        "I(650)",
        "I(max)-I(650)",
        "I(400)",
        "I(max)-I(400)",
        "coef A",
        "height",
        "weight",
        "ang",
    ],
    folder="results/1",
    cv_count=len(data_table),
    # repForStdCalc=1,
)

In [ ]:
print(data_table)